In [13]:

import numpy as np
import time
import pandas as pd
from os import listdir
from os.path import isfile, isdir, join
from PIL import Image
import cv2
import math
import random

### 訓練集 img&label

In [14]:
##將cvs裡面答案轉為陣列
df_t = pd.read_excel("./Training-label.xlsx",header=None)  
#print(df_t)
#df_t = df_t.T
df_temp = pd.get_dummies(df_t[0]) #One-hot編碼

y_t = np.array(df_temp)
print(y_t)
##將驗證圖片讀檔，二值化並存成2維陣列180*(4096)
size = 128
img_tra = []

# 指定要列出所有檔案的目錄
mypath = "./Training"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
# 以迴圈處理
j=0

for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.resize(cv2.imread(fullpath,0),(size,size))
        image = cv2.resize(cv2.imread(fullpath),(size,size))
        image_array=np.asarray(image,dtype='float32')

        img_tra.append(image_array)
        j+=1
img_tra = np.array(img_tra)
print(img_tra.shape)
print(y_t.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(180, 128, 128, 3)
(180, 18)


### 驗證/測試集img&label

In [15]:
##將cvs裡面答案轉為陣列
df_v = pd.read_excel('./Validation-label.xlsx',header=None)  
#print(df_v)
#df_v = df_v.T
df_v_temp = pd.get_dummies(df_v[0]) #One-hot編碼
y_v = np.array(df_v_temp)


img_val = []#np.zeros((50,size*size,3))
# 指定要列出所有檔案的目錄
mypath = "./Validation"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
# 以迴圈處理
j=0

for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.resize(cv2.imread(fullpath,0),(size,size))
        image = cv2.resize(cv2.imread(fullpath),(size,size))
        img_val.append(image)
img_val = np.array(img_val)

print(img_val.shape)
print(y_v.shape)

(50, 128, 128, 3)
(50, 18)


In [18]:
from keras.utils import np_utils
from keras.datasets import mnist
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential
img_pix=size
#建立一個線性堆疊的模型
model = Sequential()
#建立卷積層1
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='same',input_shape=(img_pix,img_pix,3),activation='relu'))
#建立池化層1。
model.add(MaxPooling2D(pool_size=(2,2)))
#建立卷積層2
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='relu'))
#建立池化層2
model.add(MaxPooling2D(pool_size=(2,2)))
#避免overfitting
model.add(Dropout(0.25))
#建立平坦層
model.add(Flatten())
#建立隱藏層，且避免overfitting
model.add(Dense(img_pix,activation='relu'))
model.add(Dropout(0.25))
#建立輸出層
model.add(Dense(18,activation='softmax'))
#設定損失函數，最佳化方法，以及評估模型等。
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

##開始訓練
train_history=model.fit(x=img_tra ,y=y_t,validation_data=(img_val,y_v),\
                        epochs=50,batch_size=100,verbose=2)
result=model.predict_classes(img_val)
print(result)

Train on 180 samples, validate on 50 samples
Epoch 1/50
 - 4s - loss: 67.9099 - accuracy: 0.0500 - val_loss: 813.5549 - val_accuracy: 0.0600
Epoch 2/50
 - 4s - loss: 912.9801 - accuracy: 0.0722 - val_loss: 314.0757 - val_accuracy: 0.0600
Epoch 3/50
 - 4s - loss: 408.0089 - accuracy: 0.0889 - val_loss: 92.3978 - val_accuracy: 0.1000
Epoch 4/50
 - 4s - loss: 149.6559 - accuracy: 0.0611 - val_loss: 28.9440 - val_accuracy: 0.1000
Epoch 5/50
 - 4s - loss: 35.9097 - accuracy: 0.0833 - val_loss: 5.7347 - val_accuracy: 0.1600
Epoch 6/50
 - 4s - loss: 5.5700 - accuracy: 0.2167 - val_loss: 4.1786 - val_accuracy: 0.2000
Epoch 7/50
 - 4s - loss: 2.9062 - accuracy: 0.3167 - val_loss: 3.2484 - val_accuracy: 0.2600
Epoch 8/50
 - 4s - loss: 1.9730 - accuracy: 0.4222 - val_loss: 2.5755 - val_accuracy: 0.3600
Epoch 9/50
 - 4s - loss: 1.3802 - accuracy: 0.5722 - val_loss: 2.4490 - val_accuracy: 0.3600
Epoch 10/50
 - 4s - loss: 1.0943 - accuracy: 0.6889 - val_loss: 2.2317 - val_accuracy: 0.4200
Epoch 11/5